In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

In [2]:
from divtel import *
#from divtel.ArrayConfig import *
from pathlib import Path

In [3]:
prod='PROD5' 
outdir='./MAGIC'
site='LaPalma'     #allowed values: LaPalma, Paranal

In [4]:
alt=70  #[30,50,70] you are setting ALTITUDE, zd = 90 - alt
az=180    #[0,180] 0--> North pointing, 180 --> South pointing 
# Automatically set pointing to be used in output filename
if az==0:
    pointing='North'
elif az==180:
    pointing='South'

In [5]:
#Setting MSTs camera according to the site Nectar --> LP, Flash --> Paranal
if site == 'LaPalma':
    camera = 'Nectar'
elif site == 'Paranal':
    camera = 'Flash'

In [6]:
# Loading the geometrical configuration of our array 
#( telescope position on the ground - (x,y,z), camera radius, focal lenght )
array = LoadConfig(f"MAGIC/layout_4LSTs_MAGIC.txt")

In [7]:
# Setting div values
divergence=[0.005,0.01]#[0.0022,0.0043,0.008,0.01135,0.01453]

In [8]:
# Names of the files containing simulation details for each telescope
# I did not made them up, you can find the correct names in the simtel config files for parallel pointing
configs=['CTA-PROD4-LST',f'CTA-PROD6-MST-{camera}Cam','CTA-PROD6-SST']
LST_config=configs[0]
MST_config=configs[1]
SST_config=configs[2]

In [9]:
#path = Path(f"{outdir}")
#path.mkdir(exist_ok=True)
#path = Path(f"{outdir}/{site}")
#path.mkdir(exist_ok=True)
#path = Path(f"{outdir}/{site}/{pointing}_pointing")
#path.mkdir(exist_ok=True)

In [10]:
# 'EXTRA' settings that will be used inside config file to make it look like parallel pointing files
if site=='LaPalma':
    geosite='La_Palma'
    atm='NORTH'
    hemisphere='North'
    array_config_variant=f"LST/{camera}Cam at CTA {hemisphere} with Alpha+ layout"
elif site =='Paranal':
    geosite=site
    atm='SOUTH'
    hemisphere='South'
    array_config_variant=f"LST/{camera}Cam/SST at CTA {hemisphere} with Alpha+ layout"

In [11]:
# Generating sim_telarray config file for the chosen pointing dir and for all div values 
for div in divergence: 
    #print(div)
    array.divergent_pointing(div = div, az = az, alt = alt)
    array.table.units = "deg"
    table = array.table
    filename = f"CTA-{prod}-{site}-MAGIC_4LSTs_div{div}_Zd{90 - alt}_{pointing}Pointing.cfg"
    with open(filename, 'w') as f:
        f.write("% What transmission option to use (see CTA-PROD4-site.cfg):\n")
        f.write(f"#define CTA_{geosite.upper()} 1\n")
        f.write(f"#define ATMOSPHERE_{atm} 1\n")
        f.write("#define NAVY_MARITIME 1\n")
        f.write("#define LOW_EXTINCTION 1\n")
        f.write("\n")
        f.write("#ifndef TELESCOPE\n")
        f.write("#   define TELESCOPE 0\n")
        f.write("#endif\n")
        f.write("\n")
        f.write('#if TELESCOPE == 0\n')
        f.write('   TELESCOPE_THETA={:.2f}\n'.format(90 - array.pointing["alt"].value))
        f.write('   TELESCOPE_PHI={:.2f}\n'.format(array.pointing["az"].value))
        f.write('\n% Global and default configuration for things missing in telescope-specific config.\n')
        f.write(f'#  include <{LST_config}.cfg>\n')
        for n, tel in enumerate(table, 1):
            #zd = 90 - tel["alt"]
            f.write('\n#elif TELESCOPE == {:d}\n'.format(n))
            if tel["focal"]==16.97:
                if tel['id'] ==1:
                    f.write(f"#  include <CTA-PROD4-MAGIC1_test.cfg>\n")
                elif tel['id'] ==2:
                    f.write(f'#  include <CTA-PROD4-MAGIC2_test.cfg>\n')
            
            elif tel["focal"]==28:
                if tel['id'] ==3 and 'LaPalma' in filename:
                    f.write(f"#  include <CTA-PROD4-LST-prototype.cfg>\n")
                else:
                    f.write(f"#  include <{LST_config}.cfg>\n")
                
            f.write("   TELESCOPE_THETA={:.2f}\n".format(90 - tel["alt"]))
            f.write("   TELESCOPE_PHI={:.2f}\n".format(360 - tel["az"]))
            f.write("\n")
        f.write("#else\n")
        f.write("   Error Invalid telescope for CTA-PROD5 Paranal baseline/extended configuration.\n")
        f.write("#endif\n")
        # Trigger conditions
        f.write("trigger_telescopes = 2 % We apply loose stereo trigger immediately\n")
        f.write("% To be more strict we need a matching array trigger definition.\n")
        f.close()